In [2]:
import pandas as pd
import numpy as np
from utils import entropy, normalize
import warnings
warnings.filterwarnings('ignore')


get_na_codes = lambda dataset, name : set(dataset[dataset[name].isna()]['股票代码'].to_list())
dataset = pd.read_csv('./data/因果机制研究全部数据.csv',dtype={'股票代码':'object'})
# 拿出制造业
dataset = dataset[dataset['行业代码'].str.startswith('C')]
dataset['行业代码'] = dataset['行业代码'].apply(lambda x: int(x[1:]))


important_columns=['wufei', 'zhaochenyv', 'yuanchun', '无形资产数字化', 'csmar', 'lishouxi', '当年独立申请的发明数量', '当年联合申请的发明数量', '当年独立获得的发明数量', '当年联合获得的发明数量', '当年独立申请专利总和', '当年联合申请专利总和', '当年独立获得专利总和', '当年联合获得专利总和', '独立申请实用新型&外观设计', '联合申请实用新型&外观设计', '独立获得实用新型&外观设计', '联合获得实用新型&外观设计','应收账款周转率B', '应付账款周转率B', '存货周转率B', '固定资产周转率B', '总资产周转率B']
dataset.dropna(subset=important_columns,inplace=True)
needs=['应收账款周转率B', '应付账款周转率B', '存货周转率B', '固定资产周转率B', '总资产周转率B']

# dataset[['股票代码','截止日期']+needs].sort_values(['股票代码','截止日期'])

In [3]:
matrix = normalize(dataset[needs],['应收账款周转率B','存货周转率B'])
# ,['应付账款周转率B', '固定资产周转率B', '总资产周转率B']
dataset['营运能力'] = dataset[needs].dot(entropy(matrix))
# 净资产收益率、资本积累率、资产负债率、总资产周转率
needs =['总资产周转率B','净资产收益率（ROE）B','资本积累率B','资产负债率']
matrix = normalize(dataset[needs],['资产负债率'])  # ,['总资产周转率B','净资产收益率（ROE）B','资本积累率B']
dataset['盈利能力'] = dataset[needs].dot(entropy(matrix))
dataset.drop(['应收账款周转率B', '应付账款周转率B', '存货周转率B', '固定资产周转率B', '总资产周转率B','总资产周转率B','资本积累率B','主营业务收入'],axis=1,inplace=True)

# 盈利能力相关
profitability_names = ['股票代码','截止日期','主业绩效','托宾Q值B','总资产净利润率(ROA)B','净资产收益率（ROE）B','投入资本回报率（ROIC）','盈利能力']
profitability = dataset[profitability_names]

digital_names=['股票代码','截止日期','wufei', 'zhaochenyv', 'yuanchun', '无形资产数字化', 'csmar', 'lishouxi']
digital = dataset[digital_names]

# 可能的中介变量创新能力
middle_names=[
    '截止日期', '股票代码', '成本费用利润率','销售期间费用率',
    '内部控制指数','供应链集成','组织韧性',
    '当年独立申请的发明数量', '当年联合申请的发明数量', '当年独立获得的发明数量', '当年联合获得的发明数量', '当年独立申请专利总和', '当年联合申请专利总和', '当年独立获得专利总和', '当年联合获得专利总和', '独立申请实用新型&外观设计', '联合申请实用新型&外观设计', '独立获得实用新型&外观设计', '联合获得实用新型&外观设计',
    '研发费用率','研发人员数量占比(%)','研发营收比(%)'
]
middle = dataset[middle_names]
# 控制变量与因变量
business_names = ['截止日期', '股票代码', '营运能力','行业代码', '董事会规模', '两职合一', '两权分离度(%)', '股权集中度4(%)', '独董比例', '资产负债率', '资本密集度', '股权性质', '上市期限', '企业规模', '上市省份']
business = dataset[business_names]

# 托宾Q值B
# 两权分离度(%)/研发人员数量占比(%)/研发营收比(%)/组织韧性/供应链集成/研发费用率
# 主营业务收入/内部控制指数/主业绩效
business

,截止日期,股票代码,营运能力,行业代码,董事会规模,两职合一,两权分离度(%),股权集中度4(%),独董比例,资产负债率,资本密集度,股权性质,上市期限,企业规模,上市省份
24,2011,000004,2.439983,27,11596.0,1.0,10.4400,42.7821,0.300000,0.294935,2.634865,0.0,20.0,19.095191,广东省
25,2012,000004,2.441005,27,12730.0,1.0,10.4400,38.3636,0.333333,0.232528,1.981379,0.0,21.0,19.077753,广东省
26,2013,000004,1.705997,27,11817.0,1.0,8.3520,40.6303,0.400000,0.403239,3.326970,0.0,22.0,19.305077,广东省
27,2014,000004,1.912770,27,12670.0,1.0,8.3520,35.7537,0.333333,0.534754,4.196595,0.0,23.0,19.639392,广东省
28,2015,000004,2.795013,27,11103.0,0.0,9.0944,51.7810,0.333333,0.590402,3.309746,0.0,24.0,19.803653,广东省
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40604,2022,873305,4.031366,33,19858.0,1.0,0.0000,79.4180,0.333333,0.162897,2.676968,0.0,0.0,19.695486,湖北省
40605,2019,873339,6.577956,38,11728.0,1.0,6.9033,82.7123,0.285714,0.460082,0.791019,0.0,-3.0,19.818025,江苏省
40606,2020,873339,6.782623,38,11728.0,1.0,6.9033,82.7123,0.285714,0.302510,0.772126,0.0,-2.0,19.764373,江苏省
40607,2021,873339,10.470137,38,11728.0,1.0,6.9033,82.7123,0.285714,0.478184,0.708358,0.0,-1.0,20.113307,江苏省


In [4]:
# 首先我们假设，数字化 能够通过 企业创新 影响 企业运营绩效
from scipy.stats.mstats import winsorize

data = dataset[[
    '截止日期', '股票代码','营运能力','行业代码', '董事会规模', '两职合一',  '股权集中度4(%)', '独董比例', '资产负债率', '资本密集度', '股权性质', '上市期限', '企业规模', '上市省份',
    '成本费用利润率','销售期间费用率',
    '当年独立申请的发明数量', '当年联合申请的发明数量', '当年独立获得的发明数量', '当年联合获得的发明数量', '当年独立申请专利总和', '当年联合申请专利总和', '当年独立获得专利总和', '当年联合获得专利总和', '独立申请实用新型&外观设计', '联合申请实用新型&外观设计', '独立获得实用新型&外观设计', '联合获得实用新型&外观设计',
    'wufei', 'zhaochenyv', 'yuanchun', '无形资产数字化', 'csmar', 'lishouxi'
]]
# '两权分离度(%)','研发费用率','研发人员数量占比(%)','研发营收比(%)'
# 存在缺失值的数据：两权分离度/研发费用率/研发人员数量占比/研发营收比

keywords=['营运能力', '董事会规模', '股权集中度4(%)', '独董比例', '资产负债率', '资本密集度', '企业规模', '成本费用利润率', '销售期间费用率', '当年独立申请的发明数量', '当年联合申请的发明数量', '当年独立获得的发明数量', '当年联合获得的发明数量', '当年独立申请专利总和', '当年联合申请专利总和', '当年独立获得专利总和', '当年联合获得专利总和', '独立申请实用新型&外观设计', '联合申请实用新型&外观设计', '独立获得实用新型&外观设计', '联合获得实用新型&外观设计']
# 是否缩尾处理
for keyword in keywords:
    # 此处判断是否空值，是的话为空，否的话进行屏蔽空值和无效值的1%和99%缩尾处理
    data[keyword] = np.where(data[keyword].isnull(),np.nan, winsorize(np.ma.masked_invalid(data[keyword]),limits=(0.01,0.01)))
data.to_csv('./data/dataset.csv',index=False,encoding='utf-8')